In [ ]:
import xml.etree.ElementTree as ET
import csv

In [6]:
def xml_to_csv(input_xml_file, output_label_csv_file, output_name_csv_file, output_labels_with_names_csv_file):
    # Parse the XML file
    tree = ET.parse(input_xml_file)
    root = tree.getroot()
    
    # Define CSV headers
    csv_header = ["ColumnA", "ColumnB", "ColumnC", "ColumnD", "ColumnE", "ColumnF", "ColumnG" ]
    labels_with_names_header = ["Label", "Name"]
    
    # Open the CSV files for writing
    with open(output_label_csv_file, 'w', newline='') as labelfile, \
         open(output_name_csv_file, 'w', newline='') as namefile, \
         open(output_labels_with_names_csv_file, 'w', newline='') as labels_with_names_file:
        
        label_writer = csv.writer(labelfile, delimiter=';')
        name_writer = csv.writer(namefile, delimiter=';')
        labels_with_names_writer = csv.writer(labels_with_names_file, delimiter=';')
        
        # Write headers
        label_writer.writerow(csv_header)
        name_writer.writerow(csv_header)  # Assuming this was intended to write the same header as label_writer
        labels_with_names_writer.writerow(labels_with_names_header)
        
        # Set to keep track of labels already written
        written_labels = set()
        
        # Function to recursively extract data categories and write to CSV
        def process_data_categories(element, parent_data=[]):
            # Extract label and name
            label = element.find('{http://soap.sforce.com/2006/04/metadata}label').text if element.find('{http://soap.sforce.com/2006/04/metadata}label') is not None else ""
            name = element.find('{http://soap.sforce.com/2006/04/metadata}name').text if element.find('{http://soap.sforce.com/2006/04/metadata}name') is not None else ""
            
            # Append to parent data
            current_data_label = parent_data + [label]
            current_data_name = parent_data + [name]
            
            # Write to CSV ensuring each level is in the correct column, but only if label hasn't been written yet
            if label not in written_labels:
                row_label = [''] * len(csv_header)  # Create a row with the same number of columns as the header
                row_name = [''] * len(csv_header)  # Create a row with the same number of columns as the header
                
                for i, val in enumerate(current_data_label):
                    if i < len(row_label):  # Ensure we do not exceed the row length
                        row_label[i] = val
                for i, val in enumerate(current_data_name):
                    if i < len(row_name):  # Ensure we do not exceed the row length
                        row_name[i] = val
                
                label_writer.writerow(row_label)
                name_writer.writerow(row_name)
                labels_with_names_writer.writerow([label, name])
                
                # Add label to set of written labels
                written_labels.add(label)
            
            # Recursively process child data categories
            for child in element.findall('{http://soap.sforce.com/2006/04/metadata}dataCategory'):
                process_data_categories(child, parent_data + [label])
        
        # Get the root label and name
        root_label = root.find('{http://soap.sforce.com/2006/04/metadata}label').text if root.find('{http://soap.sforce.com/2006/04/metadata}label') is not None else ""
        root_name = root.find('{http://soap.sforce.com/2006/04/metadata}name').text if root.find('{http://soap.sforce.com/2006/04/metadata}name') is not None else ""
        
        # Start processing from the root data categories
        for data_category in root.findall('.//{http://soap.sforce.com/2006/04/metadata}dataCategory'):
            process_data_categories(data_category, [root_label])


In [ ]:
# Specify the paths directly
xml_path = '/**************'
output_label_csv_path = '/**************'
output_name_csv_path = '/**************'
output_labels_with_names_csv_path = '/**************'

# Call the function directly with specified paths
xml_to_csv(xml_path, output_label_csv_path, output_name_csv_path, output_labels_with_names_csv_path)